# Imports

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
import seaborn as sns
import tabulate
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.optimizers import Adam,SGD, RMSprop,AdamW
from sklearn.preprocessing import RobustScaler,LabelEncoder
import os
from sklearn.svm import SVR

# Get Paths to datasets (Credits to Adham)

In [2]:
plt.style.use("fivethirtyeight")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/financial-risk-for-loan-approval/Loan.csv
/kaggle/input/financial-risk-for-loan-approval/CSV Generation.py
/kaggle/input/regression-test-file/Regression_test_file.csv


In [3]:
loan = pd.read_csv("/kaggle/input/financial-risk-for-loan-approval/Loan.csv")
testFile = pd.read_csv("/kaggle/input/regression-test-file/Regression_test_file.csv")
copy = loan.copy()

# Handling Missing and duplicates

In [4]:
copy.describe()

,Age,AnnualIncome,CreditScore,Experience,LoanAmount,LoanDuration,NumberOfDependents,MonthlyDebtPayments,CreditCardUtilizationRate,NumberOfOpenCreditLines,...,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,39.752600,59161.473550,571.612400,17.522750,24882.867800,54.057000,1.517300,454.292700,0.286381,3.023350,...,4891.715521,0.799918,5.002650,7.229432e+04,0.239124,0.239110,911.607052,0.402182,0.239000,50.766780
std,11.622713,40350.845168,50.997358,11.316836,13427.421217,24.664857,1.386325,240.507609,0.159793,1.736161,...,3296.771598,0.120665,2.236804,1.179200e+05,0.035509,0.042205,674.583473,0.338924,0.426483,7.778262
min,18.000000,15000.000000,343.000000,0.000000,3674.000000,12.000000,0.000000,50.000000,0.000974,0.000000,...,1250.000000,0.259203,0.000000,1.000000e+03,0.130101,0.113310,97.030193,0.016043,0.000000,28.800000
25%,32.000000,31679.000000,540.000000,9.000000,15575.000000,36.000000,0.000000,286.000000,0.160794,2.000000,...,2629.583333,0.727379,3.000000,8.734750e+03,0.213889,0.209142,493.763700,0.179693,0.000000,46.000000
50%,40.000000,48566.000000,578.000000,17.000000,21914.500000,48.000000,1.000000,402.000000,0.266673,3.000000,...,4034.750000,0.820962,5.000000,3.285550e+04,0.236157,0.235390,728.511452,0.302711,0.000000,52.000000
75%,48.000000,74391.000000,609.000000,25.000000,30835.000000,72.000000,2.000000,564.000000,0.390634,4.000000,...,6163.000000,0.892333,6.000000,8.882550e+04,0.261533,0.265532,1112.770759,0.509214,0.000000,56.000000
max,80.000000,485341.000000,712.000000,61.000000,184732.000000,120.000000,5.000000,2919.000000,0.917380,13.000000,...,25000.000000,0.999433,16.000000,2.603208e+06,0.405029,0.446787,10892.629520,4.647657,1.000000,84.000000


In [5]:
copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ApplicationDate             20000 non-null  object 
 1   Age                         20000 non-null  int64  
 2   AnnualIncome                20000 non-null  int64  
 3   CreditScore                 20000 non-null  int64  
 4   EmploymentStatus            20000 non-null  object 
 5   EducationLevel              20000 non-null  object 
 6   Experience                  20000 non-null  int64  
 7   LoanAmount                  20000 non-null  int64  
 8   LoanDuration                20000 non-null  int64  
 9   MaritalStatus               20000 non-null  object 
 10  NumberOfDependents          20000 non-null  int64  
 11  HomeOwnershipStatus         20000 non-null  object 
 12  MonthlyDebtPayments         20000 non-null  int64  
 13  CreditCardUtilizationRate   200

In [6]:
copy.isna().sum()

ApplicationDate               0
Age                           0
AnnualIncome                  0
CreditScore                   0
EmploymentStatus              0
EducationLevel                0
Experience                    0
LoanAmount                    0
LoanDuration                  0
MaritalStatus                 0
NumberOfDependents            0
HomeOwnershipStatus           0
MonthlyDebtPayments           0
CreditCardUtilizationRate     0
NumberOfOpenCreditLines       0
NumberOfCreditInquiries       0
DebtToIncomeRatio             0
BankruptcyHistory             0
LoanPurpose                   0
PreviousLoanDefaults          0
PaymentHistory                0
LengthOfCreditHistory         0
SavingsAccountBalance         0
CheckingAccountBalance        0
TotalAssets                   0
TotalLiabilities              0
MonthlyIncome                 0
UtilityBillsPaymentHistory    0
JobTenure                     0
NetWorth                      0
BaseInterestRate              0
Interest

In [7]:
copy.duplicated().sum()

0

**Dropping**

In [8]:
copy = copy.drop(columns = "ApplicationDate")
copy.drop_duplicates()
copy.dropna()

,Age,AnnualIncome,CreditScore,EmploymentStatus,EducationLevel,Experience,LoanAmount,LoanDuration,MaritalStatus,NumberOfDependents,...,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved,RiskScore
0,45,39948,617,Employed,Master,22,13152,48,Married,2,...,3329.000000,0.724972,11,126928,0.199652,0.227590,419.805992,0.181077,0,49.0
1,38,39709,628,Employed,Associate,15,26045,48,Single,1,...,3309.083333,0.935132,3,43609,0.207045,0.201077,794.054238,0.389852,0,52.0
2,47,40724,570,Employed,Bachelor,26,17627,36,Married,2,...,3393.666667,0.872241,6,5205,0.217627,0.212548,666.406688,0.462157,0,52.0
3,58,69084,545,Employed,High School,34,37898,96,Single,1,...,5757.000000,0.896155,5,99452,0.300398,0.300911,1047.506980,0.313098,0,54.0
4,37,103264,594,Employed,Associate,17,9184,36,Married,1,...,8605.333333,0.941369,5,227019,0.197184,0.175990,330.179140,0.070210,1,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,44,30180,587,Employed,High School,19,24521,36,Married,3,...,2515.000000,0.826217,1,55327,0.216021,0.195574,905.767712,0.627741,0,55.0
19996,56,49246,567,Employed,Associate,33,25818,36,Married,5,...,4103.833333,0.816618,3,64002,0.227318,0.199168,958.395633,0.334418,0,54.0
19997,44,48958,645,Employed,Bachelor,20,37033,72,Married,3,...,4079.833333,0.887216,3,103663,0.229533,0.226766,945.427454,0.357227,0,45.0
19998,60,41025,560,Employed,High School,36,14760,72,Married,3,...,3418.750000,0.843787,5,10600,0.249760,0.264873,411.168284,0.408678,0,59.0


**Dropping**

In [9]:
testFile = testFile.drop(columns = "ApplicationDate")
testFile.drop_duplicates()
testFile.dropna()

,Age,AnnualIncome,CreditScore,EmploymentStatus,EducationLevel,Experience,LoanAmount,LoanDuration,MaritalStatus,NumberOfDependents,...,TotalLiabilities,MonthlyIncome,UtilityBillsPaymentHistory,JobTenure,NetWorth,BaseInterestRate,InterestRate,MonthlyLoanPayment,TotalDebtToIncomeRatio,LoanApproved
0,45,39948,617,Employed,Master,22,13152,48,Married,2,...,19183,3329.000000,0.724972,11,126928,0.199652,0.227590,419.805992,0.181077,0
1,33,29096,570,Employed,Associate,12,40138,24,Married,3,...,50013,2424.666667,0.760758,4,36237,0.230138,0.248601,2139.408949,1.164865,0
2,50,60992,663,Employed,Bachelor,30,24642,48,Single,0,...,13350,5082.666667,0.917148,4,38852,0.188142,0.216449,771.630413,0.282456,0
3,43,100032,524,Employed,Master,21,19105,60,Married,0,...,4301,8336.000000,0.831494,6,45588,0.262105,0.209684,516.514890,0.160810,1
4,33,35882,620,Employed,Master,11,9259,48,Single,0,...,91262,2990.166667,0.883372,3,8266,0.194259,0.200011,281.760387,0.180177,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,45,45453,518,Employed,Master,23,36807,24,Married,2,...,14772,3787.750000,0.887467,5,39580,0.252807,0.237099,1940.701755,0.645159,0
996,26,47517,533,Employed,Associate,3,12185,72,Divorced,3,...,34408,3959.750000,0.799362,5,47114,0.260685,0.296527,363.851501,0.125223,0
997,69,38715,670,Employed,High School,46,11703,12,Single,3,...,4102,3226.250000,0.850983,4,189145,0.141703,0.162197,1063.040355,0.470218,0
998,23,101892,482,Employed,Bachelor,1,38161,24,Widowed,1,...,24493,8491.000000,0.823695,5,10427,0.272161,0.278276,2091.248768,0.264780,0


# Handling Categorical Columns For Dataset File

In [10]:
listOfColumns = []

listOfCategoricalColumns = []
for i in copy.select_dtypes(include = 'number'):
    listOfColumns.append(i)


for j in copy.columns:

    if j not in listOfColumns:
        listOfCategoricalColumns.append(j)
le = LabelEncoder()
for i in listOfCategoricalColumns:
    copy[i] = le.fit_transform(copy[i])

# Features and target defining

In [11]:
X_Loan = copy.drop(columns = ['RiskScore'])
Y_Loan = copy["RiskScore"]

# Handling Outliars

In [12]:
robust = RobustScaler()
scaledXLoan = robust.fit_transform(X_Loan)
scaledYLoan = robust.fit_transform(Y_Loan.values.reshape(-1,1))

# Handling Categorical Columns For Test File

In [13]:
listOfColumns1 = []

listOfCategoricalColumns1 = []
for i in testFile.select_dtypes(include = 'number'):
    listOfColumns1.append(i)


for j in testFile.columns:

    if j not in listOfColumns1:
        listOfCategoricalColumns1.append(j)
le = LabelEncoder()
for i in listOfCategoricalColumns1:
    testFile[i] = le.fit_transform(testFile[i])

# Handling Outliars

In [14]:
robust_TestFile = RobustScaler()
scaledTestFile = robust_TestFile.fit_transform(testFile)

In [15]:
scaledXLoan.shape

(20000, 34)

In [16]:
scaledYLoan.shape

(20000, 1)

In [17]:
scaledTestFile.shape

(1000, 34)

# Train test split

In [18]:
X_train, X_val, y_train, y_val = train_test_split( scaledXLoan, scaledYLoan, test_size=0.2, random_state=42)

# Neural Network 

***Training and Evaluating***

**Trial 1**
* We have used 1 hidden layer with 100 neorons.
* Adam optimizer was used.
* We tested with relu activation function.
* We used learning rate 0.001
* Finally we tested with 10 epochs.
* The MSE was approx. 0.018.

In [19]:
# ANN = Sequential()
# ANN.add(Dense(100, activation='relu', input_dim = X_train.shape[1]))
# ANN.add(Dense(1))
# ANN.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error' ,metrics=['accuracy'])
# ANN.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
# ANN.evaluate(X_val, y_val)

**Trial 2**
* We have used 1 hidden layer with 100 neorons.
* Qdam Optimizer was used.
* We tested with relu activation function.
* Learning rate with 0.005.
* Finally we tested with 50 epochs.
* The MSE was approx. 0.009.
Obsevations:
    * When incresing learning rate and no.of epochs whe MSE Decresed.  

In [20]:
# ANN = Sequential()
# ANN.add(Dense(100, activation='relu', input_dim = X_train.shape[1]))
# ANN.add(Dense(1))
# ANN.compile(optimizer=Adam(learning_rate=0.005), loss='mean_squared_error' ,metrics=['accuracy'])
# ANN.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))
# ANN.evaluate(X_val, y_val)

**Trial 3**
* We have used 1 hidden layer with 100 neorons.
* Learning rate was 0.01
* Activation function relu was applied.
* epochs number was 60
* The MSE was 0.013
Observations:
    * When highly increasing learning rate the loss has increased again.

In [21]:
# ANN = Sequential()
# ANN.add(Dense(100, activation='relu', input_dim = X_train.shape[1]))
# ANN.add(Dense(1))
# ANN.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error' ,metrics=['accuracy'])
# ANN.fit(X_train, y_train, epochs=60, batch_size=32, validation_data=(X_val, y_val))
# ANN.evaluate(X_val, y_val)

**Trial 4**
* We have used 1 hidden layer with 100 neorons.
* Learning rate was 0.009
* Activation function relu was applied.
* epochs number was 60
* The MSE was 0.0115
* Observations:
    * When slightly decreasing learning rate the loss has decreasd again.

In [22]:
# ANN = Sequential()
# ANN.add(Dense(100, activation='relu', input_dim = X_train.shape[1]))
# ANN.add(Dense(1))
# ANN.compile(optimizer=Adam(learning_rate=0.009), loss='mean_squared_error')
# ANN.fit(X_train, y_train, epochs=60, batch_size=32, validation_data=(X_val, y_val))
# ANN.evaluate(X_val, y_val)

**Trial 5**
* We have used 1 hidden layer with 100 neorons.
* Learning rate was 0.005
* Activation function tanh was applied.
* epochs number was 10
* The MSE was 0.013
* Observations:
    * When slightly decreasing learning rate the loss has decreasd again.

In [23]:
ANN = Sequential()
ANN.add(Dense(100, activation='tanh', input_dim = X_train.shape[1]))
ANN.add(Dense(1))
ANN.compile(optimizer=Adam(learning_rate=0.005), loss='mean_squared_error')
ANN.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
ANN.evaluate(X_val, y_val)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1271 - val_loss: 0.0324
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0312 - val_loss: 0.0280
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0251 - val_loss: 0.0264
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0226 - val_loss: 0.0206
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0179 - val_loss: 0.0158
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0160 - val_loss: 0.0176
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0148 - val_loss: 0.0150
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0148 - val_loss: 0.0145
Epoch 9/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0139 - val_loss: 0.0167
Epoch 10/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0145 - val_loss: 0.0174
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0171


0.017418930307030678

**Predictions**

In [24]:
predections_Scaled = ANN.predict(scaledTestFile)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


**Getting unScaled Values for predictions**

In [25]:
predictions_Inversed = robust.inverse_transform(predections_Scaled.reshape(-1,1))
predictions_Inversed = predictions_Inversed.reshape(-1)
predictions_Inversed = [round(x,1) for x in predictions_Inversed]

# File Export (Credits to Adham)

In [26]:
dictt = {
    "ID": [i for i in range(len(predictions_Inversed))],
    "Class": predictions_Inversed
}
df = pd.DataFrame(dictt)
df.to_csv("Submission.csv", index=False)